# Weighted boxes fusion: Ensembling boxes from different object detection models

![image.png](attachment:e9af3ba4-2755-402f-9e93-e2d767b7aa0a.png)

https://github.com/ZFTurbo/Weighted-Boxes-Fusion

Coordinates for boxes expected to be normalized e.g in range [0; 1]. Order: x1, y1, x2, y2.

Example of boxes ensembling for 2 models below.

First model predicts 5 boxes, second model predicts 4 boxes.
Confidence scores for each box model 1: [0.9, 0.8, 0.2, 0.4, 0.7]
Confidence scores for each box model 2: [0.5,0.3]
Labels (classes) for each box model 1: [0, 1]
Labels (classes) for each box model 2: [1, 1]
We set weight for 1st model to be 2, and weight for second model to be 1.
We set intersection over union for boxes to be match: iou_thr = 0.5
We skip boxes with confidence lower than skip_box_thr = 0.0001

In [ ]:
def wbf(result_cbnet, result_yolo, IOU_WBF=0.2):
    boxes_list = [[],[]]
    scores_list = [[], []]
    labels_list = [[], []]
    weights = [1, 1]
    iou_thr = 0.5
    skip_box_thr = 0.3
    #sigma = 0.1
    for cbnet in result_cbnet[0]:
        x1, y1, x2, y2, conf = cbnet

        boxes_list[0].append([x1/1280, y1/720, x2/1280, y2/720])
        scores_list[0].append(conf)
        labels_list[0].append(0)
    for idx, row in result_yolo.pandas().xyxy[0].iterrows():

        boxes_list[1].append([row.xmin/1280, row.ymin/720, row.xmax/1280, row.ymax/720])
        scores_list[1].append(row.confidence)
        labels_list[1].append(0)
    
    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    length = len(scores)
    res = ''
    for idx in range(length):
        conf = scores[idx]
        xmin, ymin, xmax, ymax = boxes[idx]
        if conf > IOU_WBF:
            res += f'{conf} {xmin*1280} {ymin*720} {(xmax-xmin)*1280} {(ymax-ymin)*720} '
    return res.strip(' ')